# Goal

sample 10 % of drug_related cases from the full_dataset on drug_related. store them in datalake gen 2.

assemble folder paths

In [ ]:
base_path = "/mnt/processed_data_criminal_case_analysis/drug_related_full_dataset/"


def list_csv_files(path):
    """
    List all CSV files recursively within a given path.
    """
    files = []
    for item in dbutils.fs.ls(path):
        if item.path.endswith(".csv"):
            files.append(item.path)
        elif item.isDir():  # If the item is a directory, recurse
            files.extend(list_csv_files(item.path))
    return files


select relevant columns, rename them into English

In [ ]:
column_mapping = {
    "原始链接": "OriginalLink",
    "案号": "CaseNumber",
    "案件名称": "CaseName",
    "法院": "Court",
    "所属地区": "Location",
    "案件类型": "CaseType",
    "审理程序": "TrialProcedure",
    "裁判日期": "JudgmentDate",
    "公开日期": "PublicationDate",
    "当事人": "PartiesInvolved",
    "案由": "CausesofAction",
    "法律依据": "LegalBasis",
    "全文": "FullText"
}


Select columns, rename, filter out CaseReasons, them sample the data.

In [ ]:
from pyspark.sql.functions import col


sample_fraction = 0.1

import re


csv_files = list_csv_files(base_path)
for file_path in csv_files:
    # Read the CSV file into a DataFrame
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    
    # Loop through each cause of action, filter, sample, and process/save
    sampled_df = df.sample(fraction=sample_fraction)  # Sample the filtered DataFrame

    # Extract year and month from file_path
    match = re.search(r'(\d{4})_(\d{2})_drug_related_judgment_data', file_path)
    if match:
        year = match.group(1)  # '2013'
        month = match.group(2)  # '01'
    else:
        raise ValueError(f"Month pattern not found in the file path: {file_path}")

    saved_file_name = f"{year}_{month}_drug_related_judgment_data"
    
    # Define the full path for saving the DataFrame
    full_path = f"/mnt/processed_data_criminal_case_analysis/drug_related_10_percent/{saved_file_name}"
    
    # Save the DataFrame
    sampled_df.write.mode("overwrite").option("header", "true").csv(full_path)
    



# Test code for the a small data set
 

In [ ]:
# checking the mount status
display(dbutils.fs.ls("/mnt/processed_data_criminal_case_analysis"))
display(dbutils.fs.ls("/mnt/processed_data_criminal_case_analysis/2013_judgment_data"))

path,name,size,modificationTime
dbfs:/mnt/processed_data_criminal_case_analysis/2013_judgment_data/,2013_judgment_data/,0,1710477564000
dbfs:/mnt/processed_data_criminal_case_analysis/testcsv.csv,testcsv.csv,23,1710463876000


---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-4495737001346625>, line 3
      1 # checking the mount status
      2 display(dbutils.fs.ls("/mnt/processed_data_criminal_case_analysis"))
----> 3 display(dbutils.fs.ls("/mnt/processed_data_criminal_case_analysis/2010_judgment_data"))

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o428.ls.
: java.io.FileNotFoundException: No such file or directory /mnt/processed_data_criminal_case_analysis/2010_judgment_data
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$2(DatabricksFileSystemV2.scala:135)
	at com.databricks.s3a.S3AExceptionUtils$.convertAWSExceptionToJavaIOExcepti

In [ ]:
from pyspark.sql.functions import col
import re

base_path = "/mnt/processed_data_criminal_case_analysis"
folder_names = ["/mnt/processed_data_criminal_case_analysis/2013_judgment_data"]

def list_csv_files(path):
    """
    List all CSV files recursively within a given path.
    """
    files = []
    for item in dbutils.fs.ls(path):
        if item.path.endswith(".csv"):
            files.append(item.path)
        elif item.isDir():  # If the item is a directory, recurse
            files.extend(list_csv_files(item.path))
    return files

# Causes of Action to filter and sample
causes_of_action = {
    
    "盗窃": "theft",
    "交通肇事": "traffic_accidents",
    "毒品": "drug_related",
    "诈骗": "fraud"
}

sample_fraction = 0.01

for folder in folder_names:
    csv_files = list_csv_files(folder)
    for file_path in csv_files:
        # Read the CSV file into a DataFrame
        df = spark.read.csv(file_path, header=True, inferSchema=True)
        
        # Select and rename the relevant columns
        df_selected_and_renamed = df.select([col(c).alias(column_mapping[c]) for c in column_mapping.keys()])
        
        # Filter to include only cases that mention "刑事" in CaseType
        df_criminal_cases = df_selected_and_renamed.filter(col("CaseType").contains("刑事"))
        
        # Loop through each cause of action, filter, sample, and process/save
        for cause, output_suffix in causes_of_action.items():
            df_filtered = df_criminal_cases.filter(col("CausesofAction").contains(cause))
            sampled_df = df_filtered.sample(fraction=sample_fraction)  # Sample the filtered DataFrame

            # Extract year and month from file_path
            match = re.search(r'(\d{4})年(\d{2})月裁判文书数据\.csv', file_path)
            if match:
                year = match.group(1)  # '2013'
                month = match.group(2)  # '01'
            else:
                raise ValueError("Month pattern not found in the file path.")

            saved_file_name = f"{year}_{month}_{output_suffix}_judgment_data"
            
            # Define the full path for saving the DataFrame
            full_path = f"/mnt/processed_data_criminal_case_analysis/{output_suffix}/{saved_file_name}"
            
            # Save the DataFrame
            sampled_df.write.mode("overwrite").option("header", "true").csv(full_path)
            

